## Agroforestry Suitability Workflow

This notebook evaluates the **climate suitability** of agroforestry systems **at two scales**:

1. **Plot-specific analysis**  
   - Focused on a given plot location (latitude, longitude, elevation).  
   - Shows how suitability for each species may change **at this exact site** under future climate scenarios.

2. **Regional analysis**  
   - Looks at the wider production landscape around the plot.  
   - Maps current and projected suitability across the region.  
   - Puts the plot’s results in the context of where the crop or species is most likely to thrive.

---

### Workflow Overview

1. **Start from a plot**  
   - Define the plot location and record the current agroforestry system (species, density, shading, yield).

2. **Gather species occurrence data**  
   - Search GBIF and/or local datasets for occurrences in a **wider training region**.  
   - Clean, filter, and spatially thin to reduce bias.  
   - Optionally remove isolated or wild points if focusing on cultivated systems.

3. **Prepare environmental predictors**  
   - Load current climate data (e.g., TerraClimate).  

4. **Generate background (absence) points**  
   - Sample across the training region, restricted to land.  
   - Ensure coverage across environmental gradients (e.g., lowlands, highlands).

5. **Train the species distribution model (SDM)**  
   - Use a Random Forest classifier with presence/absence data.  
   - Evaluate performance (AUC, sensitivity, specificity).  
   - Determine a probability threshold for suitability.

6. **Predict suitability**  
   - For **current** climate conditions.  
   - For **future** climate scenarios.

7. **Analyse results**  
   - **Plot scale:** Extract suitability values for the exact plot location and compare now vs future.  
   - **Regional scale:** Map suitability categories (e.g., suitable both, newly suitable, no longer suitable) across the training region.

---

**Goal:**  
Provide decision‑makers with:
- **Actionable insights for their plot** (current system resilience, alternative scenarios).  
- **Regional context** to understand how local changes fit into broader production trends.


In [1]:
import pandas as pd
from config import DATA_DIR

# Path to your cleaned observation file
obs_path = DATA_DIR / "species_location/species_occurrences_combined_cleaned.csv"

# Read file
df_obs = pd.read_csv(obs_path)

# Normalise column names
rename_map = {
    "species": "species_query", "scientificName": "species_query",
    "Longitude": "decimalLongitude", "lon": "decimalLongitude", "lng": "decimalLongitude",
    "Latitude": "decimalLatitude",   "lat": "decimalLatitude",
}
for k, v in rename_map.items():
    if k in df_obs.columns and v not in df_obs.columns:
        df_obs = df_obs.rename(columns={k: v})

# Drop rows with no species info
df_obs = df_obs.dropna(subset=["species_query"])

# Get unique species
unique_species = sorted(df_obs["species_query"].unique())

# Show as list
for sp in unique_species:
    print(sp)

print(f"\nTotal unique species: {len(unique_species)}")


Annona muricata
Artocarpus altilis
Byrsonima crassifolia
Cajanus cajan
Carica papaya
Cedrela odorata
Citrus aurantium
Citrus sinensis
Coffea arabica
Cordia alliodora
Erythrina poeppigiana
Gliricidia sepium
Inga edulis
Inga jinicuil
Inga vera
Macadamia integrifolia
Mangifera indica
Musa paradisiaca
Persea americana
Pouteria sapota
Psidium guajava
Spondias mombin
Swietenia macrophylla
Theobroma cacao

Total unique species: 24


In [2]:
import numpy as np
import pandas as pd

def clean_occurrences(df, max_isolate_distance_deg=0.5, thinning_deg=2.5/60):
    """
    Clean occurrence points per species by:
    1. Removing isolated points farther than `max_isolate_distance_deg` from all others.
    2. Applying spatial thinning so that only 1 point is kept per grid cell of size `thinning_deg`.

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame with columns: species (or species_query), decimalLongitude, decimalLatitude.
    max_isolate_distance_deg : float
        Maximum allowed distance (in degrees) for a point to have a neighbour. 
        Points farther than this from all others are removed.
    thinning_deg : float
        Grid cell size for thinning, in degrees. Default 2.5 arc‑min (~0.0416667°).

    Returns
    -------
    pd.DataFrame
        Cleaned DataFrame of occurrences.
    """

    def remove_isolated_points(species_df):
        coords = species_df[["decimalLongitude", "decimalLatitude"]].to_numpy()
        keep_idx = []
        for i, c in enumerate(coords):
            dists = np.sqrt((coords[:,0] - c[0])**2 + (coords[:,1] - c[1])**2)
            dists[i] = np.inf  # ignore self
            if np.any(dists < max_isolate_distance_deg):
                keep_idx.append(i)
        return species_df.iloc[keep_idx]

    def thin_points(species_df):
        # Assign grid cell indices
        grid_x = np.floor(species_df["decimalLongitude"] / thinning_deg)
        grid_y = np.floor(species_df["decimalLatitude"] / thinning_deg)
        species_df = species_df.assign(grid_x=grid_x, grid_y=grid_y)
        # Randomly sample 1 point per grid cell
        thinned = species_df.groupby(["grid_x", "grid_y"], group_keys=False).apply(
            lambda g: g.sample(1, random_state=42)
        )
        return thinned.drop(columns=["grid_x", "grid_y"])

    cleaned_species_list = []
    for species, group in df.groupby("species_query" if "species_query" in df.columns else "species"):
        # 1️⃣ Remove extreme isolates
        non_isolated = remove_isolated_points(group)
        # 2️⃣ Apply spatial thinning
        thinned = thin_points(non_isolated)
        print(f"{species}: {len(group)} → {len(thinned)} after cleaning")
        cleaned_species_list.append(thinned)

    return pd.concat(cleaned_species_list, ignore_index=True)


# --- Example use ---
df = clean_occurrences(
    df_obs,
    max_isolate_distance_deg=0.1,   # ~11 km
    thinning_deg=2.5/60             # 2.5 arc-min (~4.6 km)
)

print(f"✅ Cleaned dataset: {len(df)} records")


Annona muricata: 914 → 457 after cleaning
Artocarpus altilis: 1108 → 558 after cleaning
Byrsonima crassifolia: 2330 → 1340 after cleaning
Cajanus cajan: 469 → 255 after cleaning
Carica papaya: 2610 → 1399 after cleaning
Cedrela odorata: 1408 → 772 after cleaning
Citrus aurantium: 256 → 103 after cleaning
Citrus sinensis: 362 → 146 after cleaning
Coffea arabica: 3561 → 1725 after cleaning
Cordia alliodora: 1272 → 615 after cleaning
Erythrina poeppigiana: 133 → 60 after cleaning
Gliricidia sepium: 1914 → 1027 after cleaning
Inga edulis: 1570 → 866 after cleaning
Inga jinicuil: 194 → 88 after cleaning
Inga vera: 1989 → 1155 after cleaning
Macadamia integrifolia: 10 → 1 after cleaning
Mangifera indica: 2018 → 1068 after cleaning
Musa paradisiaca: 487 → 224 after cleaning
Persea americana: 1878 → 1072 after cleaning
Pouteria sapota: 476 → 225 after cleaning
Psidium guajava: 3034 → 1693 after cleaning
Spondias mombin: 820 → 373 after cleaning
Swietenia macrophylla: 550 → 230 after cleaning
T

In [3]:
occ_all = df.rename(columns={"species_query":"species",
    "decimalLongitude": "lon",
    "decimalLatitude": "lat",
})[["species", "lon", "lat"]].dropna()
occ_all

,species,lon,lat
0,Annona muricata,-106.682167,24.415475
1,Annona muricata,-106.423805,23.193857
2,Annona muricata,-106.429925,23.232595
3,Annona muricata,-106.423988,23.260112
4,Annona muricata,-106.443430,23.299000
...,...,...,...
15948,Theobroma cacao,-60.906423,14.601630
15949,Theobroma cacao,-60.859124,14.513195
15950,Theobroma cacao,-60.585957,11.254710
15951,Theobroma cacao,-60.549420,11.316900


In the following cell, we force some observation points from the agrosystems that we generated. As we need to make sure that those are suitable today.
those would ideally correspond to true observations.

In [4]:
from pathlib import Path

# ====================================================
# Expert occurrence points (from Excel + manual input)
# ====================================================
data_dir = Path("agroforestry_systems")
excel_files = sorted(data_dir.glob("*.xlsx"))

# --- Manual expert cacao points ---
expert_cacao = pd.DataFrame({
    "species": ["Theobroma cacao"] * 3,
    "lon": [-71.125, -71.228, -71.606],
    "lat": [19.505, 19.602, 19.337],
    "source": "manual"
})

# --- Manual expert coffee points ---
expert_coffee = pd.DataFrame({
    "species": ["Coffea arabica"] * 4,
    "lon": [-71.243, -71.778, -70.981, -70.960],
    "lat": [19.271, 19.404, 19.237, 19.670],
    "source": "manual"
})

# --- Collect expert species/locations from Excel files ---
expert_from_excel = []
for file_path in excel_files:
    try:
        df = pd.read_excel(file_path, sheet_name="present")
    except ValueError:
        continue

    # Standardise species names
    if "Scientific name" in df.columns:
        sp_col = "Scientific name"
    else:
        sp_col = "Species"

    tmp = df.loc[df[sp_col].notna(), [sp_col, "Longitude", "Latitude"]].rename(
        columns={sp_col: "species", "Longitude": "lon", "Latitude": "lat"}
    )
    tmp["source"] = f"excel:{file_path.stem}"
    expert_from_excel.append(tmp)

# Merge everything
expert_occ = pd.concat([expert_cacao, expert_coffee] + expert_from_excel, ignore_index=True)

# ====================================================
# Jiggle generator (species-specific)
# ====================================================
def generate_jiggle_points(base_points, n_per_point=10, jitter_deg_default=0.1):
    jiggle_list = []
    for _, row in base_points.iterrows():
        # Coffee/cacao: altitude sensitive, keep tighter jitter
        if row["species"] in ["Coffea arabica", "Theobroma cacao"]:
            jitter_deg = jitter_deg_default
        else:
            jitter_deg = jitter_deg_default  # could broaden if wanted

        for _ in range(n_per_point):
            lon_j = row["lon"] + np.random.uniform(-jitter_deg, jitter_deg)
            lat_j = row["lat"] + np.random.uniform(-jitter_deg, jitter_deg)
            jiggle_list.append({
                "species": row["species"],
                "lon": lon_j,
                "lat": lat_j,
                "source": "jiggle"
            })
    return pd.DataFrame(jiggle_list)

# Generate jiggles for all expert species
jiggles = generate_jiggle_points(expert_occ, n_per_point=5, jitter_deg_default=0.1)

# ====================================================
# Final training set
# ====================================================
occ_all = pd.concat([occ_all, expert_occ, jiggles], ignore_index=True)

print(f"✅ Final training set: {len(occ_all)} points across {occ_all['species'].nunique()} species")
occ_all.head()


✅ Final training set: 18395 points across 31 species


,species,lon,lat,source
0,Annona muricata,-106.682167,24.415475,NaN
1,Annona muricata,-106.423805,23.193857,NaN
2,Annona muricata,-106.429925,23.232595,NaN
3,Annona muricata,-106.423988,23.260112,NaN
4,Annona muricata,-106.443430,23.299000,NaN


### Building the training dataset for suitability modelling

In this step, we prepare the **input dataset** that will be used to train the suitability model for each target species.

**Workflow:**

1. **Load climate predictors**
   - Open the `bio_stack` from TerraClimate (1990–2014 baseline).
   - This contains gridded bioclimatic variables such as temperature, precipitation, PET, AET, and solar radiation.
   - The file is an `xarray.Dataset`, which allows fast spatial extraction.

2. **Define modelling extent**
   - Specify the bounding box (`bbox`) for the study region.
   - This ensures we only extract presence and background points from within the relevant spatial domain.

3. **Select predictors**
   - Choose which climate variables to use for model training.
   - Selected variables can be adjusted depending on the research question and collinearity checks.

4. **Build the training dataset**  
   - Call `build_training_dataset()`:
     - Filters the presence dataset (`occ_all`) to the study area.
     - Generates background (pseudo‑absence) points within the study area.
     - Extracts climate predictor values for all points from `bio_stack`.
     - Combines presence (label = 1) and background (label = 0) into a single DataFrame.

5. **Result**
   - The resulting `data_all` contains:
     - Geographic coordinates.
     - Selected climate predictor values.
     - Presence/absence labels.
     - Species identifiers.
   - This is the core dataset for training species distribution models.

**Notes:**
- The `background_ratio` parameter controls how many background points are generated relative to the number of presences (here: 3×).
- This step **does not yet fit the model** — it only prepares the inputs.
- The presence dataset (`occ_all`) should be cleaned before this step (e.g., remove duplicates, isolates, and apply spatial thinning if needed).


In [ ]:
from utils_suitability_modelling import build_training_dataset

import xarray as xr

# 📂 Path to your climate predictor stack
bio_path = DATA_DIR / "terra_climate/SuitabilityVariables_1990_2014.nc"

# 1️⃣ Open dataset
bio_stack = xr.open_dataset(bio_path)




# 📋 Inputs
bbox = (-115, -50, 10, 25)  # adjust to your study area

selected_predictors = [
    "MeanDiurnalRange",
    "Isothermality",
    "PrecSeasonality",
    "AnnualPET",
    "MeanTempDriestQuarter",
    "PrecDriestMonth",
    "PrecWettestMonth",
    "AnnualAET",
    "AnnualDeficit",
    "MeanSRAD",
    "MeanTempWettestQuarter",
    "PrecWarmestQuarter",
    "PrecColdestQuarter",
]
#bio_stack = bio_stack.rename({"lon": "y", "lat": "x"})

# 📂 Run
data_all = build_training_dataset(
    presence_df=occ_all,       # df with ['species', 'lon', 'lat']
    bio_stack=bio_stack,       # xarray.Dataset of your climate variables
    bbox=bbox,
    selected_predictors=selected_predictors,
    background_ratio=3
)

data_all.head()


✅ Found 18395 presence points in study area.


/Users/szelie/python_projects/biofincas_climate_risk/utils_suitability_modelling.py:75: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  land = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))


🚀 Querying predictors for 46100 points…


In [ ]:
import matplotlib.pyplot as plt


lon_min, lon_max, lat_min, lat_max = bbox

# 📋 Separate presence & absence
presences = data_all[data_all["presence"] == 1]
absences  = data_all[data_all["presence"] == 0]

plt.figure(figsize=(10, 6))

# Background points (absence)
plt.scatter(
    absences["lon"], absences["lat"],
    color="grey", alpha=0.5, s=10, label="Absence (background)"
)

# Presence points
plt.scatter(
    presences["lon"], presences["lat"],
    color="green", alpha=0.9, s=20, marker='^', label="Presence"
)

# Study area bbox
plt.plot(
    [lon_min, lon_max, lon_max, lon_min, lon_min],
    [lat_min, lat_min, lat_max, lat_max, lat_min],
    color="black", linestyle="--", linewidth=1, label="Study area"
)

plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.legend()
plt.xlim(lon_min - 1, lon_max + 1)
plt.ylim(lat_min - 1, lat_max + 1)
plt.grid(True, alpha=0.3)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import geopandas as gpd

# --- Dominican Republic bounding box ---
dr_bbox = (-72.1, -68.2, 17.4, 19.9)  # (min_lon, max_lon, min_lat, max_lat)
lon_min, lon_max, lat_min, lat_max = dr_bbox

# --- Filter for coffee presences and background absences in DR ---
dr_points = data_all[
    ((data_all["species"] == "Theobroma cacao") | (data_all["species"] == "background")) &
    (data_all["lon"] >= lon_min) & (data_all["lon"] <= lon_max) &
    (data_all["lat"] >= lat_min) & (data_all["lat"] <= lat_max)
]

# --- Separate presence & absence ---
presences = dr_points[dr_points["presence"] == 1]
absences  = dr_points[dr_points["presence"] == 0]

# --- Load world borders ---
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

# --- Plot ---
fig, ax = plt.subplots(figsize=(10, 6))

# Country borders
world.boundary.plot(ax=ax, color="black", linewidth=0.8)

# Absence points
ax.scatter(
    absences["lon"], absences["lat"],
    color="grey", alpha=0.5, s=10, label="Absence (background)"
)

# Presence points
ax.scatter(
    presences["lon"], presences["lat"],
    color="green", alpha=0.9, s=20, marker='^', label="Presence"
)

# DR bbox outline
ax.plot(
    [lon_min, lon_max, lon_max, lon_min, lon_min],
    [lat_min, lat_min, lat_max, lat_max, lat_min],
    color="black", linestyle="--", linewidth=1, label="DR bbox"
)

ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
ax.set_title("Presence & Absence points for Coffea arabica — Dominican Republic")
ax.legend()
ax.set_xlim(lon_min - 0.5, lon_max + 0.5)
ax.set_ylim(lat_min - 0.5, lat_max + 0.5)
ax.grid(True, alpha=0.3)

plt.show()


## --> our points are quite sparce for the DR, we would need more observations ideally

## Species Suitability Modelling with Random Forest

The following cell trains **Random Forest** models to predict habitat suitability for multiple species based on environmental predictors.  
It uses **presence/background points**, applies filtering to remove correlated and redundant predictors, and evaluates model performance.

### Workflow
1. **Input Data**
   - `data_all` must contain:
     - `species` — species name
     - `presence` — binary presence/background label (`1` = presence, `0` = background)
     - `lon`, `lat` — coordinates in decimal degrees
     - Environmental predictors matching `selected_predictors`

2. **Settings**
   - `species_to_plot` — species for which to plot feature importance
   - `min_distance_deg` — minimum distance between presence and background points
   - `corr_threshold` — maximum Pearson correlation allowed
   - `vif_threshold` — maximum Variance Inflation Factor allowed
   - `n_estimators` — number of trees in the Random Forest
   - `min_importance` — minimum RF importance to retain predictors

3. **Feature Selection**
   - **Correlation filter**: remove highly correlated predictors
   - **VIF filter**: remove predictors with high multicollinearity
   - **Low-importance filter**: remove features with low RF importance

4. **Model Training**
   - Filter background points to be at least `min_distance_deg` from presence points
   - Split into train/test sets (70/30, stratified)
   - Train Random Forest
   - Retrain after removing low-importance predictors

5. **Evaluation**
   - Compute **AUC** (Area Under ROC Curve)
   - Determine optimal threshold for **max sensitivity + specificity**
   - Plot feature importance for selected species

6. **Outputs**
   - AUC scores per species
   - Feature importance plots
   - Suitability predictions for each point
   - Dictionaries of:
     - `trained_classifiers`
     - `feature_columns` used per species
     - `results_by_species`
   - `df_all_results` with predictions for all species


In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.inspection import permutation_importance
from scipy.spatial import cKDTree
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import requests
from shapely.geometry import Point

# =========================================
# SETTINGS
# =========================================
species_to_plot = ["Coffea arabica", "Theobroma cacao"]
min_distance_deg = 0.1
corr_threshold = 0.85
vif_threshold = 10
n_estimators = 300
min_importance = 0.01

selected_predictors = [
    "MeanDiurnalRange", "Isothermality", "PrecSeasonality", "AnnualPET",
    "MeanTempDriestQuarter", "PrecDriestMonth", "PrecWettestMonth", "AnnualAET",
    "AnnualDeficit", "MeanSRAD", "MeanTempWettestQuarter",
    "PrecWarmestQuarter", "PrecColdestQuarter"
]

# Expert base points (from you)
coffee_points = [
    Point(-71.243, 19.271),
    Point(-71.778, 19.404),
    Point(-70.981, 19.237),
    Point(-70.960, 19.670),
]
cacao_points = [
    Point(-71.125, 19.505),
    Point(-71.228, 19.602),
    Point(-71.606, 19.337),
]

EXPERT_POINTS = {
    "Coffea arabica": coffee_points,
    "Theobroma cacao": cacao_points,
}

EXPECTED_ALT = {
    "Coffea arabica": (600, 2000),
    "Theobroma cacao": (0, 800),
}

# =========================================
# UTILS
# =========================================
def get_elevation(lat, lon):
    """Fetch elevation for one coordinate from Open-Elevation API."""
    url = "https://api.open-elevation.com/api/v1/lookup"
    params = {"locations": f"{lat},{lon}"}
    r = requests.get(url, params=params, timeout=10)
    r.raise_for_status()
    return r.json()["results"][0]["elevation"]

def jitter_points(points, n_extra=3, jitter_deg=0.05):
    """Create extra points around expert seeds with random jitter."""
    out = []
    for p in points:
        out.append(p)  # keep original
        for _ in range(n_extra):
            lon = p.x + np.random.uniform(-jitter_deg, jitter_deg)
            lat = p.y + np.random.uniform(-jitter_deg, jitter_deg)
            out.append(Point(lon, lat))
    return out

def drop_correlated(df, predictors, threshold=0.85):
    corr = df[predictors].corr().abs()
    upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
    to_drop = [col for col in upper.columns if any(upper[col] > threshold)]
    keep = [p for p in predictors if p not in to_drop]
    return keep, to_drop

def calculate_vif(df, predictors):
    X = df[predictors].dropna()
    vif_data = pd.DataFrame()
    vif_data["feature"] = predictors
    vif_data["VIF"] = [sm.OLS(X[col], sm.add_constant(X.drop(columns=[col]))).fit().rsquared for col in X.columns]
    vif_data["VIF"] = 1 / (1 - vif_data["VIF"])
    return vif_data

def drop_high_vif(df, predictors, threshold=10):
    vif_data = calculate_vif(df, predictors)
    to_drop = vif_data.loc[vif_data["VIF"] > threshold, "feature"].tolist()
    keep = [p for p in predictors if p not in to_drop]
    return keep, to_drop, vif_data

def max_sens_spec_threshold(y_true, y_prob):
    thresholds = np.linspace(0, 1, 101)
    best_thresh = 0.5
    best_score = -1
    for thr in thresholds:
        y_pred = (y_prob >= thr).astype(int)
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        sens = tp / (tp + fn) if tp + fn > 0 else 0
        spec = tn / (tn + fp) if tn + fp > 0 else 0
        score = sens + spec
        if score > best_score:
            best_score = score
            best_thresh = thr
    return best_thresh

# =========================================
# TRAINING LOOP
# =========================================
auc_results = {}
results_by_species = {}
trained_classifiers = {}
feature_columns = {}

for species in data_all["species"].unique():
    if species == "background":
        continue

    df_presence = data_all[(data_all["species"] == species) & (data_all["presence"] == 1)].copy()
    df_background_all = data_all[data_all["presence"] == 0].copy()

    if len(df_presence) < 10:
        continue

    # print(f"\n🌱 Training model for {species} ({len(df_presence)} presences)")

    # # --- Inject jittered expert presences ---
    # if species in EXPERT_POINTS:
    #     jittered = jitter_points(EXPERT_POINTS[species], n_extra=20, jitter_deg=0.5)
    #     min_alt, max_alt = EXPECTED_ALT[species]
    #     injected = []
    #     for pt in jittered:
    #         try:
    #             elev = get_elevation(pt.y, pt.x)
    #             if elev is not None and min_alt <= elev <= max_alt:
    #                 df_point = data_all.loc[
    #                     (np.isclose(data_all["lat"], pt.y, atol=0.01)) &
    #                     (np.isclose(data_all["lon"], pt.x, atol=0.01))
    #                 ].copy()
    #                 if not df_point.empty:
    #                     df_point = df_point.iloc[[0]].copy()
    #                     df_point["species"] = species
    #                     df_point["presence"] = 1
    #                     df_point["elevation"] = elev
    #                     injected.append(df_point)
    #                     print(f"➕ Added expert presence for {species} at {pt} (elev={elev} m)")
    #         except Exception as e:
    #             print(f"⚠️ Could not fetch elevation for {species} at {pt}: {e}")
    #     if injected:
    #         df_presence = pd.concat([df_presence] + injected, ignore_index=True)

    # --- Background filtering ---
    pres_coords = df_presence[["lon", "lat"]].to_numpy()
    tree = cKDTree(pres_coords)
    bg_far = []
    for _, row in df_background_all.iterrows():
        dist, _ = tree.query([row["lon"], row["lat"]], k=1)
        if dist >= min_distance_deg:
            bg_far.append(row)
    df_background = pd.DataFrame(bg_far)

    # --- Combine data ---
    df = pd.concat([df_presence, df_background], ignore_index=True)

    # --- Correlation filtering ---
    predictors_filtered, dropped_corr = drop_correlated(df, selected_predictors, corr_threshold)
    if dropped_corr:
        print(f"📉 Dropped correlated predictors: {dropped_corr}")

    # --- VIF filtering ---
    predictors_filtered, dropped_vif, vif_df = drop_high_vif(df, predictors_filtered, vif_threshold)
    if dropped_vif:
        print(f"📉 Dropped high-VIF predictors: {dropped_vif}")

    X = df[predictors_filtered]
    y = df["presence"]

    # --- Train/Test split ---
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42, stratify=y
    )

    # --- Train RF ---
    clf = RandomForestClassifier(n_estimators=n_estimators, random_state=42, n_jobs=-1)
    clf.fit(X_train, y_train)

    # --- Drop low-importance features and retrain ---
    importances = pd.Series(clf.feature_importances_, index=predictors_filtered)
    low_imp = importances[importances < min_importance].index.tolist()
    if low_imp:
        print(f"📉 Dropping low-importance features: {low_imp}")
        predictors_filtered = [p for p in predictors_filtered if p not in low_imp]
        X_train = X_train[predictors_filtered]
        X_test = X_test[predictors_filtered]
        clf.fit(X_train, y_train)

    # --- Store classifier ---
    trained_classifiers[species] = clf
    feature_columns[species] = predictors_filtered

    # --- AUC on holdout ---
    y_pred_prob = clf.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, y_pred_prob)
    auc_results[species] = auc
    print(f"🌳 AUC (holdout): {auc:.3f}")

    # --- Threshold ---
    threshold = max_sens_spec_threshold(y_test, y_pred_prob)
    print(f"🎯 Threshold (max sens+spec): {threshold:.2f}")

    # --- Cross-validation AUC ---
    cv_auc = []
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    for train_idx, test_idx in cv.split(X, y):
        X_tr, X_te = X.iloc[train_idx][predictors_filtered], X.iloc[test_idx][predictors_filtered]
        y_tr, y_te = y.iloc[train_idx], y.iloc[test_idx]
        model_cv = RandomForestClassifier(n_estimators=n_estimators, random_state=42, n_jobs=-1)
        model_cv.fit(X_tr, y_tr)
        prob_te = model_cv.predict_proba(X_te)[:, 1]
        cv_auc.append(roc_auc_score(y_te, prob_te))

    print(f"🔁 Cross-val AUC: {np.mean(cv_auc):.3f} ± {np.std(cv_auc):.3f}")

    # --- Permutation importance ---
    perm = permutation_importance(clf, X_test, y_test, n_repeats=5, random_state=42, n_jobs=-1)
    sorted_idx = perm.importances_mean.argsort()

    plt.figure(figsize=(8, 5))
    plt.barh(np.array(predictors_filtered)[sorted_idx], perm.importances_mean[sorted_idx])
    plt.xlabel("Mean Permutation Importance")
    plt.title(f"Permutation Importance — {species}")
    plt.tight_layout()
    plt.show()

    # --- Save predictions ---
    df["suitability"] = clf.predict_proba(X[predictors_filtered])[:, 1]
    df["predicted_presence"] = (df["suitability"] >= threshold).astype(int)
    results_by_species[species] = df

    # --- Feature importance plot ---
    if species in species_to_plot:
        importances = pd.Series(clf.feature_importances_, index=predictors_filtered).sort_values()
        importances.plot.barh()
        plt.title(f"Feature Importance — {species}")
        plt.tight_layout()
        plt.show()

# =========================================
# SUMMARY
# =========================================
print("\n🎉 AUC summary:")
for k, v in auc_results.items():
    print(f"{k}: {v:.3f}")

df_all_results = pd.concat(results_by_species.values(), ignore_index=True)


In [ ]:
# we addded a couple of points because the region of Centro Naturaleza was not covered with observations:

In [ ]:
import matplotlib.pyplot as plt

species = "Theobroma cacao"
df_obs = results_by_species[species]

plt.figure(figsize=(7,6))

# background points
plt.scatter(df_obs.loc[df_obs["presence"]==0, "lon"],
            df_obs.loc[df_obs["presence"]==0, "lat"],
            c="lightgrey", s=10, alpha=0.5, label="Background")

# presence points
plt.scatter(df_obs.loc[df_obs["presence"]==1, "lon"],
            df_obs.loc[df_obs["presence"]==1, "lat"],
            c="darkgreen", s=30, label="Presence")

# predicted suitable
plt.scatter(df_obs.loc[df_obs["predicted_presence"]==1, "lon"],
            df_obs.loc[df_obs["predicted_presence"]==1, "lat"],
            facecolors="none", edgecolors="red", s=40, label="Predicted suitable")



plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.legend()
plt.title(f"Observations and Predictions — {species}")

# 🌍 Zoom in to DR
plt.xlim(-72.2, -68.3)
plt.ylim(17.4, 20.1)

plt.show()


## Predicting Suitability Maps for Current and Future Climate

This cell uses the **trained Random Forest models** to predict species suitability  
across the study area for **current** and **future** climate conditions.

### Steps
1. **Prepare data**
   - Use `df_now` (current climate) and `df_future` (future climate).
   - Drop rows with missing predictor values for each species’ selected features.

2. **Generate predictions**
   - For each trained classifier (`trained_classifiers`):
     - Extract the relevant predictors (`feature_columns[species]`).
     - Predict suitability **for each grid cell**.

3. **Estimate prediction uncertainty**
   - Run predictions for each individual decision tree in the Random Forest.
   - Compute:
     - **Mean suitability** (`suitability`)
     - **Standard deviation** across trees (`suitability_std`)

4. **Save results**
   - Store both mean and std for:
     - `suitability_maps_now[species]` — current climate
     - `suitability_maps_future[species]` — future climate
   - Each entry contains:
     - `lon`, `lat`
     - Mean suitability score
     - Prediction uncertainty (std)

### Output
- `suitability_maps_now`: dict of DataFrames with suitability & uncertainty for each species (current climate).
- `suitability_maps_future`: same as above for future climate.


In [ ]:
import pandas as pd
import numpy as np
import xarray as xr

def load_and_crop_climate_data(nc_path, bbox):
    """
    Load a NetCDF climate dataset, ensure lat/lon ordering,
    crop to bounding box, and return as a DataFrame without NaNs.

    Parameters
    ----------
    nc_path : str
        Path to the NetCDF file.
    bbox : tuple
        (min_lon, max_lon, min_lat, max_lat) bounding box.

    Returns
    -------
    pd.DataFrame
        Climate data for the selected bounding box.
    """
    # 📂 Load dataset
    ds = xr.open_dataset(nc_path)

    # 🔄 Ensure lat ascending
    if ds.lat.values[0] > ds.lat.values[-1]:
        ds = ds.reindex(lat=list(reversed(ds.lat)))

    # 🔄 Ensure lon in -180..180 range
    if ds.lon.max() > 180:
        ds = ds.assign_coords(lon=((ds.lon + 180) % 360) - 180)
    ds = ds.sortby("lon")

    # ✂️ Crop dataset to bbox
    lat_min, lat_max = sorted([bbox[2], bbox[3]])
    lon_min, lon_max = sorted([bbox[0], bbox[1]])
    ds_crop = ds.sel(lon=slice(lon_min, lon_max), lat=slice(lat_min, lat_max))

    # 🚿 Drop all-NaN grid cells
    ds_crop = ds_crop.dropna(dim="lat", how="all").dropna(dim="lon", how="all")

    # 📊 Convert to DataFrame
    df = ds_crop.to_dataframe().reset_index().dropna()

    return df

# 📍 Define bounding box
bbox = (-100.98, -68.0, 13.0, 21.98)

# 📄 Load current climate
df_now = load_and_crop_climate_data(
    "/Users/szelie/data/unu/terra_climate/SuitabilityVariables_1990_2014.nc",
    bbox
)

# 📄 Load future climate
df_future = load_and_crop_climate_data(
    "/Users/szelie/data/unu/terra_climate_scenarios_ncss/plus2C/SuitabilityVariables_plus2C_1990_2014.nc",
    bbox
)

# 📄 Load future climate
df_4c = load_and_crop_climate_data(
    "/Users/szelie/data/unu/terra_climate_scenarios_ncss/plus4C/SuitabilityVariables_plus4C_1990_2014.nc",
    bbox
)

print(f"✅ df_now: {len(df_now)} grid points, df_future: {len(df_future)} grid points")


In [ ]:
df_now

In [ ]:
suitability_maps_now = {}
suitability_maps_future = {}
suitability_maps_4c = {}  # 4°C scenario

for species in trained_classifiers:
    clf = trained_classifiers[species]
    features = feature_columns[species]

    # Drop NaNs
    df_now_valid    = df_now.dropna(subset=features).copy()
    df_future_valid = df_future.dropna(subset=features).copy()
    df_4c_valid     = df_4c.dropna(subset=features).copy()

    # Prepare design matrices as NumPy (avoids the warning)
    X_now    = df_now_valid[features].to_numpy()
    X_future = df_future_valid[features].to_numpy()
    X_4c     = df_4c_valid[features].to_numpy()

    # NOW
    tree_preds_now = np.stack([tree.predict_proba(X_now)[:, 1] for tree in clf.estimators_], axis=1)
    df_now_valid["suitability"] = tree_preds_now.mean(axis=1)
    df_now_valid["suitability_std"] = tree_preds_now.std(axis=1)
    suitability_maps_now[species] = df_now_valid[["lon", "lat", "suitability", "suitability_std"]]

    # FUTURE (2°C)
    tree_preds_future = np.stack([tree.predict_proba(X_future)[:, 1] for tree in clf.estimators_], axis=1)
    df_future_valid["suitability"] = tree_preds_future.mean(axis=1)
    df_future_valid["suitability_std"] = tree_preds_future.std(axis=1)
    suitability_maps_future[species] = df_future_valid[["lon", "lat", "suitability", "suitability_std"]]

    # 4°C
    tree_preds_4c = np.stack([tree.predict_proba(X_4c)[:, 1] for tree in clf.estimators_], axis=1)
    df_4c_valid["suitability"] = tree_preds_4c.mean(axis=1)
    df_4c_valid["suitability_std"] = tree_preds_4c.std(axis=1)
    suitability_maps_4c[species] = df_4c_valid[["lon", "lat", "suitability", "suitability_std"]]


In [ ]:
import os

out_dir = str(DATA_DIR / "suitability")
os.makedirs(out_dir, exist_ok=True)

def save_suitability_dict(suit_dict, label):
    for species, df in suit_dict.items():
        fname = f"{label}_{species.replace(' ', '_')}.parquet"
        df.to_parquet(os.path.join(out_dir, fname), index=False)

save_suitability_dict(suitability_maps_now, "now")
save_suitability_dict(suitability_maps_future, "future")
save_suitability_dict(suitability_maps_4c, "4c")


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from shapely.geometry import Point

# --- Species ---
species = 'Persea americana'

# --- Threshold from prevalence ---
coffee_threshold = results_by_species[species]["predicted_presence"].mean()

# --- Get suitability for NOW, 2°C, and 4°C ---
df_now = suitability_maps_now[species].copy()
df_2c = suitability_maps_future[species].copy()
df_4c = suitability_maps_4c[species].copy()

# --- Rename columns ---
df_now = df_now.rename(columns={"suitability": "suit_now"})
df_2c = df_2c.rename(columns={"suitability": "suit_2c"})
df_4c = df_4c.rename(columns={"suitability": "suit_4c"})

# --- Merge with NOW ---
merge_2c = df_now.merge(df_2c[["lon", "lat", "suit_2c"]], on=["lon", "lat"], how="inner")
merge_4c = df_now.merge(df_4c[["lon", "lat", "suit_4c"]], on=["lon", "lat"], how="inner")

# --- Classify points ---
def classify_suitability(row, col_future):
    if row["suit_now"] >= coffee_threshold and row[col_future] >= coffee_threshold:
        return "Suitable both"
    elif row["suit_now"] < coffee_threshold and row[col_future] >= coffee_threshold:
        return "Newly suitable"
    elif row["suit_now"] >= coffee_threshold and row[col_future] < coffee_threshold:
        return "No longer suitable"
    else:
        return "Unsuitable both"

merge_2c["category"] = merge_2c.apply(lambda row: classify_suitability(row, "suit_2c"), axis=1)
merge_4c["category"] = merge_4c.apply(lambda row: classify_suitability(row, "suit_4c"), axis=1)

# --- Convert to GeoDataFrames ---
merge_2c["geometry"] = [Point(xy) for xy in zip(merge_2c["lon"], merge_2c["lat"])]
merge_4c["geometry"] = [Point(xy) for xy in zip(merge_4c["lon"], merge_4c["lat"])]

gdf_2c = gpd.GeoDataFrame(merge_2c, geometry="geometry", crs="EPSG:4326")
gdf_4c = gpd.GeoDataFrame(merge_4c, geometry="geometry", crs="EPSG:4326")

# --- Colors ---
category_colors = {
    "Unsuitable both": "lightgrey",
    "Newly suitable": "blue",
    "No longer suitable": "red",
    "Suitable both": "green",
}

# --- Create figure with two maps ---
fig, axs = plt.subplots(1, 2, figsize=(16, 6), sharex=True, sharey=True)

for ax, gdf, title in zip(
    axs,
    [gdf_2c, gdf_4c],
    [f"{species}: 2°C scenario", f"{species}: 4°C scenario"]
):
    for category, color in category_colors.items():
        subset = gdf[gdf["category"] == category]
        if not subset.empty:
            subset.plot(ax=ax, color=color, markersize=1, alpha=0.7)
        else:
            ax.plot([], [], color=color)  # Dummy for legend

    ax.set_title(f"{title}")
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")

# --- Custom legend (same for both) ---
legend_handles = [
    mlines.Line2D([], [], color=color, marker='o', linestyle='None',
                  markersize=5, label=category)
    for category, color in category_colors.items()
]
axs[1].legend(handles=legend_handles, title="Category", loc="lower right")

plt.tight_layout()
plt.show()


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from shapely.geometry import Point

# --- Species ---
species = "Coffea arabica"

# --- Threshold from prevalence ---
coffee_threshold = results_by_species[species]["predicted_presence"].mean()

# --- Get suitability for NOW, 2°C, and 4°C ---
df_now = suitability_maps_now[species].copy()
df_2c = suitability_maps_future[species].copy()
df_4c = suitability_maps_4c[species].copy()

# --- Rename columns ---
df_now = df_now.rename(columns={"suitability": "suit_now"})
df_2c = df_2c.rename(columns={"suitability": "suit_2c"})
df_4c = df_4c.rename(columns={"suitability": "suit_4c"})

# --- Merge with NOW ---
merge_2c = df_now.merge(df_2c[["lon", "lat", "suit_2c"]], on=["lon", "lat"], how="inner")
merge_4c = df_now.merge(df_4c[["lon", "lat", "suit_4c"]], on=["lon", "lat"], how="inner")

# --- Classify points ---
def classify_suitability(row, col_future):
    if row["suit_now"] >= coffee_threshold and row[col_future] >= coffee_threshold:
        return "Suitable both"
    elif row["suit_now"] < coffee_threshold and row[col_future] >= coffee_threshold:
        return "Newly suitable"
    elif row["suit_now"] >= coffee_threshold and row[col_future] < coffee_threshold:
        return "No longer suitable"
    else:
        return "Unsuitable both"

merge_2c["category"] = merge_2c.apply(lambda row: classify_suitability(row, "suit_2c"), axis=1)
merge_4c["category"] = merge_4c.apply(lambda row: classify_suitability(row, "suit_4c"), axis=1)

# --- Convert to GeoDataFrames ---
merge_2c["geometry"] = [Point(xy) for xy in zip(merge_2c["lon"], merge_2c["lat"])]
merge_4c["geometry"] = [Point(xy) for xy in zip(merge_4c["lon"], merge_4c["lat"])]

gdf_2c = gpd.GeoDataFrame(merge_2c, geometry="geometry", crs="EPSG:4326")
gdf_4c = gpd.GeoDataFrame(merge_4c, geometry="geometry", crs="EPSG:4326")

# --- Colors ---
category_colors = {
    "Unsuitable both": "lightgrey",
    "Newly suitable": "blue",
    "No longer suitable": "red",
    "Suitable both": "green",
}

# --- Create figure with two maps ---
fig, axs = plt.subplots(1, 2, figsize=(16, 6), sharex=True, sharey=True)

for ax, gdf, title in zip(
    axs,
    [gdf_2c, gdf_4c],
    [f"{species}: 2°C scenario", f"{species}: 4°C scenario"]
):
    for category, color in category_colors.items():
        subset = gdf[gdf["category"] == category]
        if not subset.empty:
            subset.plot(ax=ax, color=color, markersize=1, alpha=0.7)
        else:
            ax.plot([], [], color=color)  # Dummy for legend

    ax.set_title(f"{title}")
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")

# --- Custom legend (same for both) ---
legend_handles = [
    mlines.Line2D([], [], color=color, marker='o', linestyle='None',
                  markersize=5, label=category)
    for category, color in category_colors.items()
]
axs[1].legend(handles=legend_handles, title="Category", loc="lower right")

plt.tight_layout()
plt.show()


In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from shapely.geometry import Point

# --- Species ---
species = 'Theobroma cacao'

# --- Threshold from prevalence ---
coffee_threshold = results_by_species[species]["predicted_presence"].mean()

# --- Get suitability for NOW, 2°C, and 4°C ---
df_now = suitability_maps_now[species].copy()
df_2c = suitability_maps_future[species].copy()
df_4c = suitability_maps_4c[species].copy()

# --- Rename columns ---
df_now = df_now.rename(columns={"suitability": "suit_now"})
df_2c = df_2c.rename(columns={"suitability": "suit_2c"})
df_4c = df_4c.rename(columns={"suitability": "suit_4c"})

# --- Merge with NOW ---
merge_2c = df_now.merge(df_2c[["lon", "lat", "suit_2c"]], on=["lon", "lat"], how="inner")
merge_4c = df_now.merge(df_4c[["lon", "lat", "suit_4c"]], on=["lon", "lat"], how="inner")

# --- Classify points ---
def classify_suitability(row, col_future):
    if row["suit_now"] >= coffee_threshold and row[col_future] >= coffee_threshold:
        return "Suitable both"
    elif row["suit_now"] < coffee_threshold and row[col_future] >= coffee_threshold:
        return "Newly suitable"
    elif row["suit_now"] >= coffee_threshold and row[col_future] < coffee_threshold:
        return "No longer suitable"
    else:
        return "Unsuitable both"

merge_2c["category"] = merge_2c.apply(lambda row: classify_suitability(row, "suit_2c"), axis=1)
merge_4c["category"] = merge_4c.apply(lambda row: classify_suitability(row, "suit_4c"), axis=1)

# --- Convert to GeoDataFrames ---
merge_2c["geometry"] = [Point(xy) for xy in zip(merge_2c["lon"], merge_2c["lat"])]
merge_4c["geometry"] = [Point(xy) for xy in zip(merge_4c["lon"], merge_4c["lat"])]

gdf_2c = gpd.GeoDataFrame(merge_2c, geometry="geometry", crs="EPSG:4326")
gdf_4c = gpd.GeoDataFrame(merge_4c, geometry="geometry", crs="EPSG:4326")

# --- Colors ---
category_colors = {
    "Unsuitable both": "lightgrey",
    "Newly suitable": "blue",
    "No longer suitable": "red",
    "Suitable both": "green",
}

# --- Create figure with two maps ---
fig, axs = plt.subplots(1, 2, figsize=(16, 6), sharex=True, sharey=True)

for ax, gdf, title in zip(
    axs,
    [gdf_2c, gdf_4c],
    [f"{species}: 2°C scenario", f"{species}: 4°C scenario"]
):
    for category, color in category_colors.items():
        subset = gdf[gdf["category"] == category]
        if not subset.empty:
            subset.plot(ax=ax, color=color, markersize=1, alpha=0.7)
        else:
            ax.plot([], [], color=color)  # Dummy for legend

    ax.set_title(f"{title}")
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")

# --- Custom legend (same for both) ---
legend_handles = [
    mlines.Line2D([], [], color=color, marker='o', linestyle='None',
                  markersize=5, label=category)
    for category, color in category_colors.items()
]
axs[1].legend(handles=legend_handles, title="Category", loc="lower right")

plt.tight_layout()
plt.show()
